In [1]:
import polars as pl

import polars.selectors as cs

In [2]:
df_movies_path = r"F:\Datasets\CSV datasets\movies\movies.csv"
df_movies_reviews_path = r"F:\Datasets\CSV datasets\movies\movies_reviews.csv"

In [3]:
df_movies = pl.read_csv(df_movies_path)
df_reviews = pl.read_csv(df_movies_reviews_path)

In [4]:
df_joined = df_movies.join(
    df_reviews,
    on="id",
    how="inner",
    suffix="_suffix",
)

In [5]:
drop_columns = [column for column in df_joined.columns if column.endswith("_suffix")]

df_joined = df_joined.select(cs.exclude(drop_columns))

In [6]:
df_joined.filter(
    pl.col('director').str.split(',').list.len() > 1
)

id,title,releaseDate,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,production_companies,director,writer,top_cast,quote,score,date,author,publicationName,review_type
i64,str,str,str,str,str,i64,str,f64,i64,str,f64,f64,str,str,str,str,str,str,i64,str,str,str,str
2000547319,"""Singin' in the Rain""","""1952-04-11""","""Passed""","""Comedy,Musical,Romance""","""A silent film production compa…",103,"""Singin' Swingin' Glorious Feel…",99.0,17,"""Universal acclaim""",87.0,313.0,"""Universal acclaim""","""Loew's,Metro-Goldwyn-Mayer (MG…","""Stanley Donen,Gene Kelly""","""Betty Comden,Adolph Green""","""Gene Kelly,Donald O'Connor,Deb…","""such a joyous and entertaining…",100,"""2018-02-18""","""back_in_1939""",null,"""user"""
2000547319,"""Singin' in the Rain""","""1952-04-11""","""Passed""","""Comedy,Musical,Romance""","""A silent film production compa…",103,"""Singin' Swingin' Glorious Feel…",99.0,17,"""Universal acclaim""",87.0,313.0,"""Universal acclaim""","""Loew's,Metro-Goldwyn-Mayer (MG…","""Stanley Donen,Gene Kelly""","""Betty Comden,Adolph Green""","""Gene Kelly,Donald O'Connor,Deb…","""You will never find more joy a…",100,"""2020-05-05""","""Dostoyevsky3""",null,"""user"""
2000547319,"""Singin' in the Rain""","""1952-04-11""","""Passed""","""Comedy,Musical,Romance""","""A silent film production compa…",103,"""Singin' Swingin' Glorious Feel…",99.0,17,"""Universal acclaim""",87.0,313.0,"""Universal acclaim""","""Loew's,Metro-Goldwyn-Mayer (MG…","""Stanley Donen,Gene Kelly""","""Betty Comden,Adolph Green""","""Gene Kelly,Donald O'Connor,Deb…","""Perhaps the best Golden Age of…",100,"""2020-12-25""","""Mglovesfun""",null,"""user"""
2000547319,"""Singin' in the Rain""","""1952-04-11""","""Passed""","""Comedy,Musical,Romance""","""A silent film production compa…",103,"""Singin' Swingin' Glorious Feel…",99.0,17,"""Universal acclaim""",87.0,313.0,"""Universal acclaim""","""Loew's,Metro-Goldwyn-Mayer (MG…","""Stanley Donen,Gene Kelly""","""Betty Comden,Adolph Green""","""Gene Kelly,Donald O'Connor,Deb…","""Super entertaining classic. Ho…",100,"""2019-10-05""","""CodyZamboni""",null,"""user"""
2000547319,"""Singin' in the Rain""","""1952-04-11""","""Passed""","""Comedy,Musical,Romance""","""A silent film production compa…",103,"""Singin' Swingin' Glorious Feel…",99.0,17,"""Universal acclaim""",87.0,313.0,"""Universal acclaim""","""Loew's,Metro-Goldwyn-Mayer (MG…","""Stanley Donen,Gene Kelly""","""Betty Comden,Adolph Green""","""Gene Kelly,Donald O'Connor,Deb…","""Glorious Performance **** far …",100,"""2019-08-29""","""TootsieWootsy""",null,"""user"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2000549063,"""Death of a Nation""","""2018-08-03""","""PG-13""","""Documentary,History""","""Not since 1860 have the Democr…",108,"""Can we save America a second t…",1.0,8,"""Overwhelming dislike""",44.0,297.0,"""Mixed or average""","""D'Souza Media,Pure Flix / Qual…","""Dinesh D'Souza,Bruce Schooley""","""Dinesh D'Souza,Bruce Schooley""","""Dinesh D'Souza,Victoria Chilap…","""D’Souza fails, as ever, to mak…",0,"""2018-07-30""","""Vadim Rizov""","""The A.V. Club""","""critic"""
2000549063,"""Death of a Nation""","""2018-08-03""","""PG-13""","""Documentary,History""","""Not since 1860 have the Democr…",108,"""Can we save America a second t…",1.0,8,"""Overwhelming dislike""",44.0,297.0,"""Mixed or average""","""D'Souza Media,Pure Flix / Qual…","""Dinesh D'Souza,Bruce Schooley""","""Dinesh D'Souza,Bruce Schooley""","""Dinesh D'Souza,Victoria Chilap…","""The meandering and insufferabl…",0,"""2018-07-30""","""David Ehrlich""","""IndieWire""","""critic"""
2000549063,"""Death of a Nation""","""2018-08-03""","""PG-13""","""Documentary,History""","""Not since 1860 have the Democr…",108,"""Can we save America a second t…",1.0,8,"""Overwhelming dislike""",44.0,297.0,"""Mixed or average""","""D'Souza Media,Pure Flix / Qual…","""Dinesh D'Souza,Bruce Schooley""","""Di

In [7]:
df_joined = df_joined.with_columns(
    pl.col('releaseDate').str.strptime(pl.Date, '%Y-%m-%d', strict=False),
    (pl.col('duration') * 60 * 1000).cast(pl.Duration('ms')),
    pl.col('metascore_count').cast(pl.UInt8),
    pl.col('date').str.strptime(pl.Date, '%Y-%m-%d', strict=False),
)

In [8]:
df_joined = df_joined.with_columns(
    pl.col('genres').str.split(',').alias('genres')
).explode('genres')

In [13]:
df_joined = df_joined.with_columns(
    pl.col('director').str.split(',')
).explode('director')

In [14]:
df_joined

id,title,releaseDate,rating,genres,description,duration,tagline,metascore,metascore_count,metascore_sentiment,userscore,userscore_count,userscore_sentiment,production_companies,director,writer,top_cast,quote,score,date,author,publicationName,review_type
i64,str,date,str,str,str,duration[ms],str,f64,u8,str,f64,f64,str,str,str,str,str,str,i64,date,str,str,str
2000545497,"""Dekalog (1988)""",1996-03-22,"""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",9h 32m,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""What a bitter pill to swallow …",100,2023-11-22,"""Siza""",null,"""user"""
2000545497,"""Dekalog (1988)""",1996-03-22,"""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",9h 32m,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""The best 10 hours you will spe…",100,2021-05-01,"""ItsMahdiyarr""",null,"""user"""
2000545497,"""Dekalog (1988)""",1996-03-22,"""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",9h 32m,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""Touching and human, Dekalog is…",100,2019-12-05,"""Andremax""",null,"""user"""
2000545497,"""Dekalog (1988)""",1996-03-22,"""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",9h 32m,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""Krzysztof Kieslowski's ""The De…",100,2018-04-26,"""thelol""",null,"""user"""
2000545497,"""Dekalog (1988)""",1996-03-22,"""TV-MA""","""Drama""","""This masterwork by Krzysztof K…",9h 32m,null,100.0,13,"""Universal acclaim""",78.0,162.0,"""Generally favorable""","""Telewizja Polska (TVP),Zespol …","""Krzysztof Kieslowski""","""Krzysztof Kieslowski,Krzysztof…","""Artur Barcis,Olgierd Lukaszewi…","""For me it is simply breathtaki…",100,2024-10-31,"""Daki1105""",null,"""user"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2000538594,"""Boogie Woogie""",2010-04-23,"""R""","""Drama""","""A comedy of manners set agains…",1h 34m,"""Seduction. Betrayal. Death. Th…",null,11,null,46.0,10.0,"""Mixed or average""","""The Works International,Consta…","""Duncan Ward""","""Danny Moynihan""","""Gillian Anderson,Alan Cumming,…","""A stinging satire of the conte…",50,2010-10-25,"""gazpacho""",null,"""user"""
2000538594,"""Boogie Woogie""",2010-04-23,"""R""","""Comedy""","""A comedy of manners set agains…",1h 34m,"""Seduction. Betrayal. Death. Th…",null,11,null,46.0,10.0,"""Mixed or average""","""The Works International,Consta…","""Duncan Ward""","""Danny Moynihan""","""Gillian Anderson,Alan Cumming,…","""If you're looking for a reason…",30,2013-03-02,"""Taplinger""",null,"""user"""
2000538594,"""Boogie Woogie""",2010-04-23,"""R""","""Drama""","""A comedy of manners set agains…",1h 34m,"""Seduction. Betrayal. Death. Th…",null,11,null,46.0,10.0,"""Mixed or average""","""The Works International,Consta…","""Duncan Ward""","""Danny Moynihan""","""Gillian Anderson,Alan Cumming,…","""If you're looking for a reason…",30,2013-03-02,"""Taplinger""",null,"""user"""
